<a href="https://colab.research.google.com/github/Chethu7781/-bhxfnr-/blob/master/pysparkINTV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# pip install pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.master('local').appName('demoApp').getOrCreate()
spark

IF we have the schema for creating a dataframe, we can use that else we can create our own schema and apply for that as well
```
new_schema = StructType([
                 StructFeild('column1', columnType),
                 StructFeild('column2', StringType()),
                 StructFeild('column3', IntegerType()),
                 structFeild('column4', BoolType()),
                  ])
```
for including the schema in the df
```
df = spark.read.csv('filename.csv', inferSchema=True, schema=new_schema, header=True).
```





In [8]:
#Reading existing csv data from hdfs
#command to check the data file in hdfs {$hdfs dfs -ls /}
# df_hdfs = spark.read.option('csv').load('hdfs://172.12.13.145:8020/{directoryofhdfscsvfile}')
df1 = spark.read.csv('/content/annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csv', header=True)

In [14]:
df1.show(5)

+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
|year|industry_code_ANZSIC|industry_name_ANZSIC|rme_size_grp|            variable|value|             unit|
+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
|2011|                   A|Agriculture, Fore...|         a_0|       Activity unit|46134|            COUNT|
|2011|                   A|Agriculture, Fore...|         a_0|Rolling mean empl...|    0|            COUNT|
|2011|                   A|Agriculture, Fore...|         a_0|Salaries and wage...|  279|DOLLARS(millions)|
|2011|                   A|Agriculture, Fore...|         a_0|Sales, government...| 8187|DOLLARS(millions)|
|2011|                   A|Agriculture, Fore...|         a_0|        Total income| 8866|DOLLARS(millions)|
+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
only showing top 5 rows



The default port no of hdfs is 5007, you can visit the site
eg: 172.12.13.145:5007
can check the data divided into how many blocks
if you have read data from hdfs, you can check the no of partitions in the code
$ **print(df_hdfs.rdd.getNumPartitions())**

In [15]:
print(df1.rdd.getNumPartitions())

1


The spark will have its default configurations while configuring we can check that using code $**print(spark.SparkContext.defaultParralelism)**

In [17]:
print(spark.sparkContext.defaultParallelism)

1


In [19]:
print(df1.schema)

StructType([StructField('year', StringType(), True), StructField('industry_code_ANZSIC', StringType(), True), StructField('industry_name_ANZSIC', StringType(), True), StructField('rme_size_grp', StringType(), True), StructField('variable', StringType(), True), StructField('value', StringType(), True), StructField('unit', StringType(), True)])


In [24]:
#To filter the dataframe we can use  where
df1.where('industry_code_ANZSIC="A"').show(5)

+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
|year|industry_code_ANZSIC|industry_name_ANZSIC|rme_size_grp|            variable|value|             unit|
+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
|2011|                   A|Agriculture, Fore...|         a_0|       Activity unit|46134|            COUNT|
|2011|                   A|Agriculture, Fore...|         a_0|Rolling mean empl...|    0|            COUNT|
|2011|                   A|Agriculture, Fore...|         a_0|Salaries and wage...|  279|DOLLARS(millions)|
|2011|                   A|Agriculture, Fore...|         a_0|Sales, government...| 8187|DOLLARS(millions)|
|2011|                   A|Agriculture, Fore...|         a_0|        Total income| 8866|DOLLARS(millions)|
+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
only showing top 5 rows



In [32]:
#To convert the datatype of column we need to cast it, we have different types
df1.withColumn('value',col('value').cast('int'))

DataFrame[year: string, industry_code_ANZSIC: string, industry_name_ANZSIC: string, rme_size_grp: string, variable: string, value: int, unit: string]

In [47]:
x = df1.withColumn('new_value', df1.value/1000)

In [50]:
x.show(5)

+----+--------------------+--------------------+------------+--------------------+-----+-----------------+---------+
|year|industry_code_ANZSIC|industry_name_ANZSIC|rme_size_grp|            variable|value|             unit|new_value|
+----+--------------------+--------------------+------------+--------------------+-----+-----------------+---------+
|2011|                   A|Agriculture, Fore...|         a_0|       Activity unit|46134|            COUNT|   46.134|
|2011|                   A|Agriculture, Fore...|         a_0|Rolling mean empl...|    0|            COUNT|      0.0|
|2011|                   A|Agriculture, Fore...|         a_0|Salaries and wage...|  279|DOLLARS(millions)|    0.279|
|2011|                   A|Agriculture, Fore...|         a_0|Sales, government...| 8187|DOLLARS(millions)|    8.187|
|2011|                   A|Agriculture, Fore...|         a_0|        Total income| 8866|DOLLARS(millions)|    8.866|
+----+--------------------+--------------------+------------+---

In [51]:
#To get the unique values in the dataframe of particular columns we use distinct
df1.select('industry_code_ANZSIC').distinct().show()

+--------------------+
|industry_code_ANZSIC|
+--------------------+
|                   K|
|                   F|
|                   Q|
|                   E|
|                   B|
|                   L|
|                   M|
|                   D|
|                   O|
|                   C|
|                   J|
|                   A|
|                   N|
|                   S|
|                   R|
|                   G|
|                   I|
|                   P|
|                 all|
|                   H|
+--------------------+



In [55]:
#creating a new column based on existing column constraints using when type
df1.withColumn('Rating', when(col('industry_code_ANZSIC')=='A', "HIGH").otherwise("LOW")).show(10)

+----+--------------------+--------------------+------------+--------------------+-----+-----------------+------+
|year|industry_code_ANZSIC|industry_name_ANZSIC|rme_size_grp|            variable|value|             unit|Rating|
+----+--------------------+--------------------+------------+--------------------+-----+-----------------+------+
|2011|                   A|Agriculture, Fore...|         a_0|       Activity unit|46134|            COUNT|  HIGH|
|2011|                   A|Agriculture, Fore...|         a_0|Rolling mean empl...|    0|            COUNT|  HIGH|
|2011|                   A|Agriculture, Fore...|         a_0|Salaries and wage...|  279|DOLLARS(millions)|  HIGH|
|2011|                   A|Agriculture, Fore...|         a_0|Sales, government...| 8187|DOLLARS(millions)|  HIGH|
|2011|                   A|Agriculture, Fore...|         a_0|        Total income| 8866|DOLLARS(millions)|  HIGH|
|2011|                   A|Agriculture, Fore...|         a_0|   Total expenditure| 7618|

Note: DataFrame is domain specific language
IF we want to use pure SQL statements in the dataframe, we can create sql table from dataframes
there are two types of table creation from dataframes
1. temporary 
2. Permananet

In [59]:
df1.createOrReplaceTempView('Temptable1')
spark.sql('select industry_code_ANZSIC, count(*) from Temptable1 group by industry_code_ANZSIC').show()

+--------------------+--------+
|industry_code_ANZSIC|count(1)|
+--------------------+--------+
|                   K|     891|
|                   F|     891|
|                   Q|     891|
|                   E|     891|
|                   B|     891|
|                   L|     891|
|                   M|     891|
|                   D|     891|
|                   O|     891|
|                   C|     891|
|                   J|     891|
|                   A|     891|
|                   N|     891|
|                   S|     891|
|                   R|     891|
|                   G|     891|
|                   I|     891|
|                   P|     891|
|                 all|      99|
|                   H|     891|
+--------------------+--------+



DEFAULT database in the spark is named **default**

In [60]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [61]:
#can create a new database 
spark.sql("create database demo")
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|     demo|
+---------+



In [62]:
#To reuse the table or data, we should create a permananent table
df1.write.saveAsTable('demo.permanent1')
spark.sql('show tables in demo').show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|     demo|permanent1|      false|
|         |temptable1|       true|
+---------+----------+-----------+

